# ElTopo.jl

ElTopo.jl is a library which allows to restructure and refine triangular mesh as it evolves. Suitable to simulate time dependant boundary problems, such as droplet behaviour uppon external fields, gradients, and it's changing surface tension. Or interfaces of homogenous liquids in a container. Full range of possibilities can be explored in the original C++ library ElTopo, which is interfaces here with Cxx.jl. 



# The Enright test

To visually see the algorithm in action we may look how a triangulated sphere behaves under periodic incompressable shear flow.

## Mesh generation

In [1]:
using GeometryTypes
### Using predefined mesh from a file

"""
`subdivide(msh::HomogenousMesh,f::Function)`

Returns a subdived triangular mesh from passed mesh `msh` and interpolator function `f`. The interpolator `f` is expected to accept two vertex indicies of the edge and to return a tuple of coordinates of the middle point (as one wishes to define it). 
"""
function subdivide(msh::HomogenousMesh,f::Function)
    ### Creating an index for the new points on the edges
    edges = filter(x->x[1]<x[2],decompose(Face{2,Int},msh))
    epoint(v1,v2) = findfirst(x->x==(v2>v1 ? Face(v1,v2) : Face(v2,v1)), edges) + maximum(maximum(msh.faces))
            
    newfaces = Face{3,Int}[]

    newvertices = copy(msh.vertices)
    resize!(newvertices,length(msh.faces) + length(edges))
    
    for (v1,v2,v3) in msh.faces

        ev3 = epoint(v1,v2)
        ev1 = epoint(v2,v3)
        ev2 = epoint(v3,v1)

        ### Usually, does assignment twice but is important if the surface is not tightly connected
        newvertices[ev3] = Point(f(v1,v2)...)
        newvertices[ev1] = Point(f(v2,v3)...)
        newvertices[ev2] = Point(f(v3,v1)...)

        push!(newfaces,Face(v1,ev3,ev2))
        push!(newfaces,Face(v2,ev1,ev3))
        push!(newfaces,Face(v3,ev2,ev1))
        push!(newfaces,Face(ev1,ev2,ev3))
    end

    return HomogenousMesh(newvertices,newfaces)
end


t = ( 1 + sqrt( 5 ) ) / 2;

vertices = Point{3,Float64}[
    [ -1,  t,  0 ], [  1, t, 0 ], [ -1, -t,  0 ], [  1, -t,  0 ],
    [  0, -1,  t ], [  0, 1, t ], [  0, -1, -t ], [  0,  1, -t ],
    [  t,  0, -1 ], [  t, 0, 1 ], [ -t,  0, -1 ], [ -t,  0,  1 ]
]

faces = Face{3,Int64}[
    [1, 12, 6], [1, 6, 2], [1, 2, 8], [1, 8, 11], [1, 11, 12], [2, 6, 10], [6, 12, 5], 
    [12, 11, 3], [11, 8, 7], [8, 2, 9], [4, 10, 5], [4, 5, 3], [4, 3, 7], [4, 7, 9],  
    [4, 9, 10], [5, 10, 6], [3, 5, 12], [7, 3, 11], [9, 7, 8], [10, 9, 2] 
]

msh = HomogenousMesh(vertices ./ sqrt(1+t^2),faces)

### Subdivide to create a sphere mesh

function sf(msh,v1,v2)
    p1 = msh.vertices[v1]
    p2 = msh.vertices[v2]
    return (p1 + p2)/norm(p1+p2) 
end

msh = subdivide(msh,(v1,v2)->sf(msh,v1,v2))
msh = subdivide(msh,(v1,v2)->sf(msh,v1,v2))


HomogenousMesh(
    faces: 320xFace{3,Int64},     vertices: 200xPoint{3,Float64}, )


## Velocity integration

In [2]:
using Makie
using ElTopo

function velocity(t,pos)
    x,y,z = pos
    
    x = x*0.15 + 0.35
    y = y*0.15 + 0.35
    z = z*0.15 + 0.35

    u = 2*sin(pi*x)^2 * sin(2*pi*y) * sin(2*pi*z) * sin(2/3*pi*t)
    v = - sin(2*pi*x) * sin(pi*y)^2 * sin(2*pi*z) * sin(2/3*pi*t)
    w = - sin(2*pi*x) * sin(2*pi*y) * sin(pi*z)^2 * sin(2/3*pi*t)

    [u,v,w] /0.3 #/0.15
end

x = Node(msh)
y = lift(x->x,x)

scene = Scene(show_axis=false)

wireframe!(scene,y,linewidth = 3f0)
mesh!(scene,y, color = :white, shading = false)

display(scene)

t = 0
Δt = 0.01
N = convert(Int,floor(pi/ Δt))

#msh = HomogenousMesh(v0,f0)
par = SurfTrack(allow_vertex_movement=true)

for i in 1:N
    @show t
    ### Second order RK2
    global v = msh.vertices
    global f = msh.faces
    
    k1 = [velocity(t,pos) for pos in v]
    v1 = v .+ k1 .* Δt
    k2 = [velocity(t+Δt/2,pos) for pos in v1]

    v2 = v .+ k2 .* Δt

    mshn = HomogenousMesh(v2,f)
    global msh = stabilize(mshn,par)

    push!(x,msh)
    
    AbstractPlotting.force_update!()
    sleep(Δt)

    global t+=Δt
end

┌ Info: Precompiling ElTopo [6379d376-cf42-11e8-13e7-47834244551c]
└ @ Base loading.jl:1186


t = 0
t = 0.01
t = 0.02
t = 0.03
t = 0.04
t = 0.05
t = 0.060000000000000005
t = 0.07
t = 0.08
t = 0.09
t = 0.09999999999999999
t = 0.10999999999999999
t = 0.11999999999999998
t = 0.12999999999999998
t = 0.13999999999999999
t = 0.15
t = 0.16
t = 0.17
t = 0.18000000000000002
t = 0.19000000000000003
t = 0.20000000000000004
t = 0.21000000000000005
t = 0.22000000000000006
t = 0.23000000000000007
t = 0.24000000000000007
t = 0.25000000000000006
t = 0.26000000000000006
t = 0.2700000000000001
t = 0.2800000000000001
t = 0.2900000000000001
t = 0.3000000000000001
t = 0.3100000000000001
t = 0.3200000000000001
t = 0.3300000000000001
t = 0.34000000000000014
t = 0.35000000000000014
t = 0.36000000000000015
t = 0.37000000000000016
t = 0.38000000000000017
t = 0.3900000000000002
t = 0.4000000000000002
t = 0.4100000000000002
t = 0.4200000000000002
t = 0.4300000000000002
t = 0.4400000000000002
t = 0.45000000000000023
t = 0.46000000000000024
t = 0.47000000000000025
t = 0.48000000000000026
t = 0.4900000000000